In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import json
import pandas as pd

# DATA PREPROCESSING

In [2]:
file = "/content/DC2021.json"#"/content/response_1718276451321.json"

In [3]:
with open(file) as train_file:
    dict_train = json.load(train_file)

In [4]:
# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(dict_train)
train.reset_index(level=0, inplace=True)

In [ ]:
train

,index,_id,user_id,local_date_str,datetime,data
0,0,650063492838e89d8060cb5e,ES0184000000051497CT0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.44}
1,1,650063492838e89d8060cb5f,ES0184000000051503CY0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.09}
2,2,650063492838e89d8060cb60,ES0184000000051504CF0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.42}
3,3,650063492838e89d8060cb61,ES0184000000051508CB0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0}
4,4,650063492838e89d8060cb62,ES0184000000051509CN0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.23}
...,...,...,...,...,...,...
4995,4995,6500634a2838e89d8060ddf3,ES0184000000051509CN0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.51}
4996,4996,6500634a2838e89d8060ddf4,ES0184000000051532KN0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.24}
4997,4997,6500634a2838e89d8060ddf5,ES0184000000051775DW0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.31}
4998,4998,6500634a2838e89d8060dde8,ES0184000000051497CT0F,2021-07-30,2021-07-30T16:00:00,{'energy_consumption_kwh': 0.13}


In [ ]:
train.user_id.nunique()

7

In [ ]:
train.datetime.nunique()

715

The measurements start from the 30th of june 2021, at 10 pm, and last until the 30th of july at 4 pm. This implies that I have complete measurements for a total of 29 days (29 * 24 = 696), then I have 17 for the last day of july and 2 for the last of june => 696 + 17+2 = 715

The measurements are evenly timestamped, and the collection has an hourly frequency.

In [5]:
dict_measurements = train.data.values

In [ ]:
dict_measurements


array([{'energy_consumption_kwh': 0.44}, {'energy_consumption_kwh': 0.09},
       {'energy_consumption_kwh': 0.42}, ...,
       {'energy_consumption_kwh': 0.31}, {'energy_consumption_kwh': 0.13},
       {'energy_consumption_kwh': 0.42}], dtype=object)

In [6]:
meter_reading = []
for el in dict_measurements:
  for k, v in el.items():
    meter_reading.append(v)

In [7]:
train['meter_reading'] = meter_reading

In [ ]:
train

,index,_id,user_id,local_date_str,datetime,data,meter_reading
0,0,650063492838e89d8060cb5e,ES0184000000051497CT0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.44},0.44
1,1,650063492838e89d8060cb5f,ES0184000000051503CY0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.09},0.09
2,2,650063492838e89d8060cb60,ES0184000000051504CF0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.42},0.42
3,3,650063492838e89d8060cb61,ES0184000000051508CB0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0},0.00
4,4,650063492838e89d8060cb62,ES0184000000051509CN0F,2021-07-01,2021-06-30T22:00:00,{'energy_consumption_kwh': 0.23},0.23
...,...,...,...,...,...,...,...
4995,4995,6500634a2838e89d8060ddf3,ES0184000000051509CN0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.51},0.51
4996,4996,6500634a2838e89d8060ddf4,ES0184000000051532KN0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.24},0.24
4997,4997,6500634a2838e89d8060ddf5,ES0184000000051775DW0F,2021-07-30,2021-07-30T15:00:00,{'energy_consumption_kwh': 0.31},0.31
4998,4998,6500634a2838e89d8060dde8,ES0184000000051497CT0F,2021-07-30,2021-07-30T16:00:00,{'energy_consumption_kwh': 0.13},0.13


In [ ]:
train.meter_reading.min(), train.meter_reading.max()

(0.0, 2.78)

In [ ]:
np.isnan(train.meter_reading).sum()

0

In [8]:
drop_cols_df = train[["user_id", "datetime", "meter_reading"]]

In [ ]:
drop_cols_df

,user_id,datetime,meter_reading
0,ES0184000000051497CT0F,2021-06-30T22:00:00,0.44
1,ES0184000000051503CY0F,2021-06-30T22:00:00,0.09
2,ES0184000000051504CF0F,2021-06-30T22:00:00,0.42
3,ES0184000000051508CB0F,2021-06-30T22:00:00,0.00
4,ES0184000000051509CN0F,2021-06-30T22:00:00,0.23
...,...,...,...
4995,ES0184000000051509CN0F,2021-07-30T15:00:00,0.51
4996,ES0184000000051532KN0F,2021-07-30T15:00:00,0.24
4997,ES0184000000051775DW0F,2021-07-30T15:00:00,0.31
4998,ES0184000000051497CT0F,2021-07-30T16:00:00,0.13


In [9]:
drop_cols_df['datetime'] = pd.to_datetime(drop_cols_df['datetime'])

In [ ]:
drop_cols_df

,user_id,datetime,meter_reading
0,ES0184000000051497CT0F,2021-06-30 22:00:00,0.44
1,ES0184000000051503CY0F,2021-06-30 22:00:00,0.09
2,ES0184000000051504CF0F,2021-06-30 22:00:00,0.42
3,ES0184000000051508CB0F,2021-06-30 22:00:00,0.00
4,ES0184000000051509CN0F,2021-06-30 22:00:00,0.23
...,...,...,...
4995,ES0184000000051509CN0F,2021-07-30 15:00:00,0.51
4996,ES0184000000051532KN0F,2021-07-30 15:00:00,0.24
4997,ES0184000000051775DW0F,2021-07-30 15:00:00,0.31
4998,ES0184000000051497CT0F,2021-07-30 16:00:00,0.13


In [10]:
sorted_df = drop_cols_df.sort_values(by = ['user_id', 'datetime'])

In [ ]:
sorted_df.user_id.unique()

array(['ES0184000000051497CT0F', 'ES0184000000051503CY0F',
       'ES0184000000051504CF0F', 'ES0184000000051508CB0F',
       'ES0184000000051509CN0F', 'ES0184000000051532KN0F',
       'ES0184000000051775DW0F'], dtype=object)

In [11]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051497CT0F"]

In [ ]:
visualization

,user_id,datetime,meter_reading
0,ES0184000000051497CT0F,2021-06-30 22:00:00,0.44
7,ES0184000000051497CT0F,2021-06-30 23:00:00,0.63
14,ES0184000000051497CT0F,2021-07-01 00:00:00,0.12
21,ES0184000000051497CT0F,2021-07-01 01:00:00,0.13
28,ES0184000000051497CT0F,2021-07-01 02:00:00,0.13
...,...,...,...
4970,ES0184000000051497CT0F,2021-07-30 12:00:00,0.85
4977,ES0184000000051497CT0F,2021-07-30 13:00:00,0.23
4984,ES0184000000051497CT0F,2021-07-30 14:00:00,0.90
4991,ES0184000000051497CT0F,2021-07-30 15:00:00,0.23


In [12]:
visualis = visualization.set_index('datetime')

In [13]:
import plotly.graph_objects as go

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
sorted_df.user_id.unique()

array(['ES0184000000051497CT0F', 'ES0184000000051503CY0F',
       'ES0184000000051504CF0F', 'ES0184000000051508CB0F',
       'ES0184000000051509CN0F', 'ES0184000000051532KN0F',
       'ES0184000000051775DW0F'], dtype=object)

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051503CY0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051504CF0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051508CB0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051509CN0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051532KN0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [ ]:
visualization = sorted_df[sorted_df.user_id == "ES0184000000051775DW0F"]
visualis = visualization.set_index('datetime')
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualis.index, y=visualis['meter_reading'], name='meter readings'))
fig.update_layout(showlegend=True, title='meter readings')
fig.show()

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sorted_df

,user_id,datetime,meter_reading
0,ES0184000000051497CT0F,2021-06-30 22:00:00,0.44
7,ES0184000000051497CT0F,2021-06-30 23:00:00,0.63
14,ES0184000000051497CT0F,2021-07-01 00:00:00,0.12
21,ES0184000000051497CT0F,2021-07-01 01:00:00,0.13
28,ES0184000000051497CT0F,2021-07-01 02:00:00,0.13
...,...,...,...
4969,ES0184000000051775DW0F,2021-07-30 11:00:00,0.47
4976,ES0184000000051775DW0F,2021-07-30 12:00:00,0.79
4983,ES0184000000051775DW0F,2021-07-30 13:00:00,0.38
4990,ES0184000000051775DW0F,2021-07-30 14:00:00,0.75


In [16]:
only_meter_df = sorted_df.drop(['datetime'], axis = 1)
only_meter_df

,user_id,meter_reading
0,ES0184000000051497CT0F,0.44
7,ES0184000000051497CT0F,0.63
14,ES0184000000051497CT0F,0.12
21,ES0184000000051497CT0F,0.13
28,ES0184000000051497CT0F,0.13
...,...,...
4969,ES0184000000051775DW0F,0.47
4976,ES0184000000051775DW0F,0.79
4983,ES0184000000051775DW0F,0.38
4990,ES0184000000051775DW0F,0.75


In [17]:
#Apply Normalization
def create_test_sequences(dataframe, time_steps):
    scaler = MinMaxScaler(feature_range=(0,1))
    output = []
    output2 = []
    for building_id, gdf in dataframe.groupby("user_id"):
       gdf[['meter_reading']] = scaler.fit_transform(gdf[['meter_reading']])
       building_data = np.array(gdf[['meter_reading']]).astype(float)
       for i in range(0, len(building_data), time_steps):
        end_ix = i + time_steps
        if end_ix > len(gdf):
          break
        output.append(building_data[i : end_ix,:])
        output2.append(building_data[i : end_ix,0])
    return np.stack(output), np.stack(output2)

In [18]:
train_window = 72

In [19]:
X_test, y_test = create_test_sequences(only_meter_df, train_window)

In [20]:
X_test.shape, y_test.shape

((63, 72, 1), (63, 72))

In [21]:
BATCH_SIZE =  128
N_EPOCHS = 40
hidden_size = 1/8

In [22]:
import torch.utils.data as data_utils

In [23]:
if torch.cuda.is_available():
    device =  torch.device('cuda')
else:
    device = torch.device('cpu')

# Convolutional Autoencoder

In [24]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self, n_features, latent_size): #(1, 32) #train_window previously: n_features
    super().__init__()
    # CONVOLUTIONAL ENCODER
    #in_channels = train_window
    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels= latent_size, kernel_size=7, padding=3, stride=2)
    self.conv2 = nn.Conv1d(in_channels=latent_size, out_channels= latent_size//2, kernel_size=7, padding=3, stride=2)
    self.conv3 = nn.Conv1d(in_channels=latent_size//2, out_channels= latent_size//4, kernel_size=7, padding=3, stride=2)
    self.relu = nn.ReLU(True)
    self.dropout = nn.Dropout(p=0.2)
  def forward(self, w):
    out = self.conv1(w.permute(0, 2, 1))  #w.permute(0, 2, 1) ---> needed because conv1d wants input in form (batch, n_features, window_size)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.conv2(out)
    out = self.relu(out)
    out = self.conv3(out)
    z = self.relu(out)
    return z

class Decoder(nn.Module):
  def __init__(self, latent_size, out_size): #(32, 1)
    super().__init__()
    self.conv1 = nn.ConvTranspose1d(latent_size//4, latent_size//2, 7, 2, 3, 1) #output_padding = 1
    self.conv3 = nn.ConvTranspose1d(latent_size//2, latent_size, 7, 2, 3, 1)
    self.conv4 = nn.ConvTranspose1d(latent_size, 1, 7, 2, 3, 1) #out_size
    self.relu = nn.ReLU(True)
    self.dropout = nn.Dropout(p=0.2)
    self.sigmoid = nn.Sigmoid()

  def forward(self, z):
    out = self.conv1(z)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.conv3(out)
    out = self.relu(out)
    out = self.conv4(out)
    w = self.sigmoid(out)
    return w.permute(0, 2, 1)

class ConvAE(nn.Module):
  def __init__(self, input_dim, latent_size): #(1, 32)
    super().__init__()
    self.encoder = Encoder(input_dim, latent_size)
    self.decoder = Decoder(latent_size, input_dim)

  def training_step(self, batch, criterion, n):
    z = self.encoder(batch)
    w = self.decoder(z)
    loss = criterion(w, batch)
    return loss

  def validation_step(self, batch, criterion, n):
    with torch.no_grad():
        z = self.encoder(batch)
        w = self.decoder(z)
        loss = criterion(w, batch)
    return loss

  def epoch_end(self, epoch, result, result_train):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result_train, result))

def evaluate(model, val_loader, criterion, device, n):
    batch_loss = []
    for [batch] in val_loader:
       batch = batch.to(device)
       loss = model.validation_step(batch, criterion, n) #, w
       batch_loss.append(loss)

    epoch_loss = torch.stack(batch_loss).mean()
    return epoch_loss


def training(epochs, model, train_loader, val_loader, device, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(list(model.encoder.parameters())+list(model.decoder.parameters()))
    # Setup loss function
    criterion = nn.MSELoss().to(device)
    for epoch in range(epochs):
        train_loss = []
        for [batch] in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            loss = model.training_step(batch, criterion, epoch+1)
            train_loss.append(loss)
            loss.backward()
            optimizer.step()
        result_train = torch.stack(train_loss).mean()

        result = evaluate(model, val_loader, criterion, device, epoch+1)
        model.epoch_end(epoch, result, result_train)
        res = result_train.item()
        history.append((res, result.item()))
    return history

def testing(model, test_loader, device):
    results=[]
    reconstruction = []
    criterion = nn.MSELoss().to(device)
    with torch.no_grad():
        for [batch] in test_loader:
            batch = batch.to(device)
            w=model.decoder(model.encoder(batch))
            #print("Batch: ", batch.size())
            #print("W: ", w.size())
            #batch_s = batch.reshape(-1, batch.size()[1] * batch.size()[2])
            #w_s = w.reshape(-1, w.size()[1] * w.size()[2])
            batch_s = batch[:, :, 0]
            batch_s = batch_s.reshape(batch.size()[0], batch.size()[1], 1)
            w_s = w
            #results.append(criterion(w, batch))
            results.append(torch.mean((batch_s-w_s)**2,axis=1))
            reconstruction.append(w)
    return results, reconstruction

In [25]:
model = ConvAE(1, 32)
model = model.to(device)

In [26]:
print(model)

ConvAE(
  (encoder): Encoder(
    (conv1): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
    (conv2): Conv1d(32, 16, kernel_size=(7,), stride=(2,), padding=(3,))
    (conv3): Conv1d(16, 8, kernel_size=(7,), stride=(2,), padding=(3,))
    (relu): ReLU(inplace=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (conv1): ConvTranspose1d(8, 16, kernel_size=(7,), stride=(2,), padding=(3,), output_padding=(1,))
    (conv3): ConvTranspose1d(16, 32, kernel_size=(7,), stride=(2,), padding=(3,), output_padding=(1,))
    (conv4): ConvTranspose1d(32, 1, kernel_size=(7,), stride=(2,), padding=(3,), output_padding=(1,))
    (relu): ReLU(inplace=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (sigmoid): Sigmoid()
  )
)


In [27]:
checkpoint = torch.load("/content/conv_ae_40_09_06.pth", map_location = torch.device('cpu'))

model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder.load_state_dict(checkpoint['decoder'])

<All keys matched successfully>

In [28]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().view(([X_test.shape[0],train_window, 1]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [29]:
results, w = testing(model, test_loader, device)

In [30]:
results

[tensor([[4.8345e-03],
         [4.4594e-03],
         [2.1054e-03],
         [4.6343e-03],
         [3.5581e-03],
         [5.1100e-03],
         [2.1620e-03],
         [2.8832e-03],
         [3.1161e-03],
         [5.2864e-03],
         [9.7888e-04],
         [1.3736e-03],
         [3.5115e-04],
         [5.5667e-04],
         [1.4882e-03],
         [4.9151e-04],
         [7.7641e-04],
         [4.5629e-04],
         [1.2666e-03],
         [6.7868e-03],
         [7.9652e-04],
         [2.3483e-03],
         [1.0124e-03],
         [8.7186e-04],
         [1.2638e-03],
         [2.7184e-03],
         [1.1228e-03],
         [8.2172e-06],
         [9.4167e-05],
         [3.4990e-04],
         [1.0418e-03],
         [8.6535e-05],
         [4.7822e-04],
         [7.9139e-04],
         [5.7918e-04],
         [6.1503e-04],
         [3.6677e-03],
         [2.6434e-03],
         [3.8068e-04],
         [3.8906e-03],
         [1.2513e-03],
         [8.4932e-04],
         [8.9874e-04],
         [3

In [ ]:
w

[tensor([[0.1093, 0.1087, 0.1078,  ..., 0.0930, 0.0921, 0.0918],
         [0.0906, 0.0818, 0.0754,  ..., 0.0759, 0.0825, 0.0891],
         [0.0816, 0.0727, 0.0672,  ..., 0.0670, 0.0671, 0.0685],
         ...,
         [0.1216, 0.1105, 0.1006,  ..., 0.1401, 0.1306, 0.1222],
         [0.1082, 0.0935, 0.0833,  ..., 0.1641, 0.1481, 0.1320],
         [0.1001, 0.0855, 0.0771,  ..., 0.0961, 0.0870, 0.0824]])]

In [31]:
w1 = w[0]

In [ ]:
w1

tensor([[0.1093, 0.1087, 0.1078,  ..., 0.0930, 0.0921, 0.0918],
        [0.0906, 0.0818, 0.0754,  ..., 0.0759, 0.0825, 0.0891],
        [0.0816, 0.0727, 0.0672,  ..., 0.0670, 0.0671, 0.0685],
        ...,
        [0.1216, 0.1105, 0.1006,  ..., 0.1401, 0.1306, 0.1222],
        [0.1082, 0.0935, 0.0833,  ..., 0.1641, 0.1481, 0.1320],
        [0.1001, 0.0855, 0.0771,  ..., 0.0961, 0.0870, 0.0824]])

In [32]:
reconstruction_test = w1.flatten().detach().cpu().numpy()

In [33]:
reconstruction_test

array([0.15710446, 0.16518013, 0.0677322 , ..., 0.10500073, 0.19874756,
       0.25656605], dtype=float32)

In [34]:
len(reconstruction_test)

4536

In [35]:
y_test1 = y_test.flatten()

In [36]:
new_df = pd.DataFrame(y_test1, columns=['y_test'])

In [37]:
new_df['pred'] = reconstruction_test

In [38]:
new_df

,y_test,pred
0,0.130112,0.157104
1,0.200743,0.165180
2,0.011152,0.067732
3,0.014870,0.017724
4,0.014870,0.013426
...,...,...
4531,0.043243,0.087115
4532,0.086486,0.089200
4533,0.091892,0.105001
4534,0.178378,0.198748


In [39]:
new_df['abs_loss'] = np.abs(new_df.y_test - new_df.pred)
new_df['rel_loss'] = np.abs((new_df['pred']-new_df['y_test'])/(1+new_df['pred'])) #1+ a denominatore

In [40]:
mre = new_df['rel_loss'].values
#Interquartile threshold
threshold = (np.percentile(np.squeeze(mre), 75)) + 1.5 *((np.percentile(np.squeeze(mre), 75))-(np.percentile(np.squeeze(mre), 25)))
new_df['threshold'] = threshold
new_df['predicted_anomaly'] = new_df['rel_loss'] > new_df['threshold']

In [41]:
vis = new_df[648:1296] # corresponding to one building: 648 points

In [42]:
predicted_anomalies = vis[vis.predicted_anomaly == 1]

In [43]:
len(predicted_anomalies)

26

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=vis.index, y=vis['y_test'], name='y_test'))
fig.add_trace(go.Scatter(x=vis.index, y=vis['pred'], name='pred'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['y_test'], mode='markers', marker=dict(color='green', size = 8), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='GT VS Predictions')
fig.show()

# Linear Autoencoder

In [45]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self, in_size, latent_size):
    super().__init__()
    self.linear1 = nn.Linear(in_size, int(in_size/2))
    self.linear2 = nn.Linear(int(in_size/2), int(in_size/4))
    self.linear3 = nn.Linear(int(in_size/4), latent_size)
    self.relu = nn.ReLU(True)

  def forward(self, w):
    out = self.linear1(w) #w
    out = self.relu(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.linear3(out)
    z = self.relu(out)
    return z

class Decoder(nn.Module):
  def __init__(self, latent_size, out_size):
    super().__init__()
    self.linear1 = nn.Linear(latent_size, int(out_size/4))
    self.linear2 = nn.Linear(int(out_size/4), int(out_size/2))
    self.linear3 = nn.Linear(int(out_size/2), out_size)
    self.relu = nn.ReLU(True)
    self.sigmoid = nn.Sigmoid()

  def forward(self, z):
    out = self.linear1(z)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.linear3(out)
    w = self.sigmoid(out)
    return w

class LinearAE(nn.Module):
  def __init__(self, w_size, z_size):
    super().__init__()
    self.encoder = Encoder(w_size, z_size)
    self.decoder = Decoder(z_size, w_size)

  def training_step(self, batch, criterion, n):
    z = self.encoder(batch)
    w = self.decoder(z)
    loss = criterion(w, batch)
    return loss

  def validation_step(self, batch, criterion, n):
    with torch.no_grad():
        z = self.encoder(batch)
        w = self.decoder(z)
        loss = criterion(w, batch)
    return loss


  def epoch_end(self, epoch, result, result_train):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result_train, result))


def evaluate(model, val_loader, criterion, device, n):
    batch_loss = []
    for [batch] in val_loader:
       batch = batch.to(device)
       loss = model.validation_step(batch, criterion, n)
       batch_loss.append(loss)

    epoch_loss = torch.stack(batch_loss).mean()
    return epoch_loss.item()


def training(epochs, model, train_loader, val_loader, device, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(list(model.encoder.parameters())+list(model.decoder.parameters()))
    criterion = nn.MSELoss().to(device)
    for epoch in range(epochs):
        train_loss = []
        for [batch] in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            #Train AE
            loss = model.training_step(batch, criterion, epoch+1)
            train_loss.append(loss)
            loss.backward()
            optimizer.step()
        result_train = torch.stack(train_loss).mean()
        result = evaluate(model, val_loader, criterion, device, epoch+1)
        model.epoch_end(epoch, result, result_train)
        res = result_train.item()
        history.append((res, result))
    return history

def testing(model, test_loader, device):
    results=[]
    reconstruction = []
    with torch.no_grad():
        for [batch] in test_loader:
            batch = batch.to(device)
            w=model.decoder(model.encoder(batch))
            results.append(torch.mean((batch-w)**2,axis=1))
            reconstruction.append(w)
    return results, reconstruction

In [46]:
test_loader1 = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().view(([X_test.shape[0],train_window]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [47]:
model1 = LinearAE(72, 9)
model1 = model1.to(device)
print(model1)

LinearAE(
  (encoder): Encoder(
    (linear1): Linear(in_features=72, out_features=36, bias=True)
    (linear2): Linear(in_features=36, out_features=18, bias=True)
    (linear3): Linear(in_features=18, out_features=9, bias=True)
    (relu): ReLU(inplace=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=9, out_features=18, bias=True)
    (linear2): Linear(in_features=18, out_features=36, bias=True)
    (linear3): Linear(in_features=36, out_features=72, bias=True)
    (relu): ReLU(inplace=True)
    (sigmoid): Sigmoid()
  )
)


In [48]:
checkpoint1 = torch.load("/content/linear_ae_40_09_06.pth", map_location = torch.device('cpu'))

model1.encoder.load_state_dict(checkpoint1['encoder'])
model1.decoder.load_state_dict(checkpoint1['decoder'])

<All keys matched successfully>

In [49]:
results1, w1 = testing(model1, test_loader1, device)

In [50]:
w11 = w1[0]

In [51]:
reconstruction_test1= w11.flatten().detach().cpu().numpy()
reconstruction_test1

array([0.10928567, 0.10871086, 0.10780796, ..., 0.09606282, 0.08695257,
       0.08238856], dtype=float32)

In [52]:
new_df1 = pd.DataFrame(y_test1, columns=['y_test'])

In [53]:
new_df1['pred_linear'] = reconstruction_test1

In [54]:
new_df1['abs_loss'] = np.abs(new_df1.y_test - new_df1.pred_linear)
new_df1['rel_loss'] = np.abs((new_df1['pred_linear']-new_df1['y_test'])/(1+new_df1['pred_linear']))

In [55]:
mre = new_df1['rel_loss'].values
#Interquartile threshold
threshold = (np.percentile(np.squeeze(mre), 75)) + 1.5 *((np.percentile(np.squeeze(mre), 75))-(np.percentile(np.squeeze(mre), 25)))
new_df1['threshold'] = threshold
new_df1['predicted_anomaly'] = new_df1['rel_loss'] > new_df1['threshold']

In [56]:
vis = new_df1[648:1296]
predicted_anomalies = vis[vis.predicted_anomaly == 1]
print(len(predicted_anomalies))
fig = go.Figure()
fig.add_trace(go.Scatter(x=vis.index, y=vis['y_test'], name='y_test'))
fig.add_trace(go.Scatter(x=vis.index, y=vis['pred_linear'], name='pred'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['y_test'], mode='markers', marker=dict(color='green', size = 8), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='GT VS Predictions')
fig.show()

24


# LSTM Autoencoder

In [57]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self, in_size, latent_size):
    super().__init__()
    self.lstm = nn.LSTM(input_size=in_size, hidden_size=latent_size, num_layers=1, batch_first=True, dropout = 0.2
            # input and output tensors are provided as (batch, seq_len, feature(size))
        )
    self.dropout = nn.Dropout(0.2)
  def forward(self, w):
    z, (h_n, c_n) = self.lstm(w)
    h_n = self.dropout(h_n)
    return h_n

class Decoder(nn.Module):
  def __init__(self, latent_size, out_size, train_window):
    super().__init__()
    self.latent_size = latent_size
    self.window = train_window
    out_size = 1
    self.lstm = nn.LSTM(input_size=latent_size, hidden_size=latent_size, num_layers=1, batch_first=True, dropout = 0.2
            # input and output tensors are provided as (batch, seq_len, feature(size))
        )
    self.dropout = nn.Dropout(0.2)
    self.output_layer = nn.Linear(latent_size, out_size)

  def forward(self, z):
    batch = z.size()[1]
    z = z.squeeze()
    input = z.repeat(1, self.window)
    input = input.reshape((batch, self.window, self.latent_size))
    w, (h_n, c_n) = self.lstm(input)
    w = self.dropout(w)
    out = self.output_layer(w)
    return out

class LstmAE(nn.Module):
  def __init__(self, input_dim, latent_size, train_window):
    super().__init__()
    self.encoder = Encoder(input_dim, latent_size)
    self.decoder = Decoder(latent_size, input_dim, train_window)

  def training_step(self, batch, criterion, n):
    z = self.encoder(batch)
    w = self.decoder(z)
    loss = criterion(w, batch)
    return loss

  def validation_step(self, batch, criterion, n):
    with torch.no_grad():
        z = self.encoder(batch)
        w = self.decoder(z)
        loss = criterion(w, batch)
    return loss

  def epoch_end(self, epoch, result, result_train):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result_train, result))

def evaluate(model, val_loader, criterion, device, n):
    batch_loss = []
    for [batch] in val_loader:
       batch = batch.to(device)
       loss = model.validation_step(batch, criterion, n)
       batch_loss.append(loss)

    epoch_loss = torch.stack(batch_loss).mean()
    return epoch_loss


def training(epochs, model, train_loader, val_loader, device, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(list(model.encoder.parameters())+list(model.decoder.parameters()))
    criterion = nn.L1Loss().to(device)
    for epoch in range(epochs):
        train_loss = []
        for [batch] in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            loss = model.training_step(batch, criterion, epoch+1)
            train_loss.append(loss)
            loss.backward()
            optimizer.step()
        result_train = torch.stack(train_loss).mean()

        result= evaluate(model, val_loader, criterion, device, epoch+1)
        model.epoch_end(epoch, result, result_train)
        res = result_train.item()
        history.append((res, result.item()))
    return history

def testing(model, test_loader, device):
    results=[]
    reconstruction = []
    criterion = nn.MSELoss().to(device)
    with torch.no_grad():
        for [batch] in test_loader:
            batch = batch.to(device)
            w=model.decoder(model.encoder(batch))
            #batch_s = batch.reshape(-1, batch.size()[1] * batch.size()[2])
            #w_s = w.reshape(-1, w.size()[1] * w.size()[2])
            #batch_s = batch[:, :, 0]
            #batch_s = batch_s.reshape(batch.size()[0], batch.size()[1], 1)
            #w_s = w
            results.append(criterion(w, batch))
            #results.append(torch.mean((batch_s-w_s)**2,axis=1))
            reconstruction.append(w)
    return results, reconstruction

In [58]:
test_loader2 = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().view(([X_test.shape[0],train_window, 1]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [59]:
model2 = LstmAE(1, 32, train_window)
model2 = model2.to(device)
print(model2)

LstmAE(
  (encoder): Encoder(
    (lstm): LSTM(1, 32, batch_first=True, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (lstm): LSTM(32, 32, batch_first=True, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
    (output_layer): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [60]:
checkpoint2 = torch.load("/content/lstm_ae_40_09_06.pth", map_location = torch.device('cpu'))

model2.encoder.load_state_dict(checkpoint2['encoder'])
model2.decoder.load_state_dict(checkpoint2['decoder'])

<All keys matched successfully>

In [61]:
results2, w2 = testing(model2, test_loader2, device)

In [62]:
w22 = w2[0]

In [63]:
reconstruction_test2= w22.flatten().detach().cpu().numpy()
reconstruction_test2

array([0.05833831, 0.07454696, 0.14127433, ..., 0.12581709, 0.11213118,
       0.1252212 ], dtype=float32)

In [64]:
new_df2 = pd.DataFrame(y_test1, columns=['y_test'])

In [65]:
new_df2['pred_lstm'] = reconstruction_test2

In [66]:
new_df2['abs_loss'] = np.abs(new_df2.y_test - new_df2.pred_lstm)
new_df2['rel_loss'] = np.abs((new_df2['pred_lstm']-new_df2['y_test'])/(1+new_df2['pred_lstm']))

In [67]:
mre = new_df2['rel_loss'].values
#Interquartile threshold
threshold = (np.percentile(np.squeeze(mre), 75)) + 1.5 *((np.percentile(np.squeeze(mre), 75))-(np.percentile(np.squeeze(mre), 25)))
new_df2['threshold'] = threshold
new_df2['predicted_anomaly'] = new_df2['rel_loss'] > new_df2['threshold']

In [69]:
vis = new_df2[648:1296]
predicted_anomalies = vis[vis.predicted_anomaly == 1]
print(len(predicted_anomalies))
fig = go.Figure()
fig.add_trace(go.Scatter(x=vis.index, y=vis['y_test'], name='y_test'))
fig.add_trace(go.Scatter(x=vis.index, y=vis['pred_lstm'], name='pred'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['y_test'], mode='markers', marker=dict(color='green', size = 8), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='GT VS Predictions')
fig.show()

15


# LSTM Forecasting

In [70]:
import torch
import torch.nn as nn
from typing import Dict, List, Tuple
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
import numpy as np
class LstmModel(nn.Module):
  def __init__(self, in_size, latent_size):
    super().__init__()
    """
    in_size: number of features in input
    latent_size: size of the latent space of the lstm
    Ex. in_size = 5, latent_size = 50
    """
    self.lstm = nn.LSTM(input_size=in_size, hidden_size=latent_size, num_layers=1, batch_first=True, dropout = 0.2
            # input and output tensors are provided as (batch, seq_len, feature(size))
        )
    self.dropout = nn.Dropout(0.2)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(latent_size, in_size)

  def forward(self, w):
    z, (h_n, c_n) = self.lstm(w)
    forecast = z[:, -1, :]
    forecast = self.relu(forecast)
    forecast = self.dropout(forecast)
    output = self.fc(forecast)
    return output


def training(epochs, model, train_loader, val_loader, device, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters())
    criterion = nn.MSELoss().to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = []
        for X_batch, y_batch in train_loader:
          X_batch = X_batch.to(device)
          y_batch = y_batch.to(device)

          z = model(X_batch)
          #print("Z: ", z.size())
          #print("Y: ", y_batch.size())
          loss = criterion(z.squeeze(), y_batch) #z.squeeze() for lead --> out_size == 1
          train_loss.append(loss)

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
        result_train = torch.stack(train_loss).mean()

        model.eval()
        batch_loss = []
        for X_batch, y_batch in val_loader:
          X_batch = X_batch.to(device)
          y_batch = y_batch.to(device)
          with torch.no_grad():
            z = model(X_batch)
            loss = criterion(z, y_batch)
          batch_loss.append(loss)

        result = torch.stack(batch_loss).mean()

        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result_train, result))


        history.append(result_train)
    return history

def testing(model, test_loader, device):
    results=[]
    forecast = []
    criterion = nn.MSELoss().to(device)
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w=model(X_batch)
            #batch_s = y_batch.reshape(-1, y_batch.size()[1] * y_batch.size()[2])
            #w_s = w.reshape(-1, w.size()[1] * w.size()[2])
            #results.append(criterion(w.squeeze(), y_batch))
            results.append(torch.mean((y_batch.unsqueeze(1)-w)**2,axis=1)) #(y_batch.unsqueeze(1) for lead
            forecast.append(w)
    return results, forecast

def testing_substitution(model, test, train_window, threshold, device):
   """
   Idea: instead of testing in the traditional way, at inference time we evaluate for each forecasted data point whether it can
   be associated to an anomaly or not. If we would predict an anomaly, we then proceed by substituting the meter_reading value
   with the predictions just made.
   """
   predictions = []
   forecasts = []
   scaler = MinMaxScaler(feature_range=(0,1))
   for building_id, gdf in test.groupby("building_id"):
      gdf[['meter_reading']] = scaler.fit_transform(gdf[['meter_reading']])
      building_data = np.array(gdf[['meter_reading']]).astype(float)
      for i in range(len(building_data)):
        # find the end of this sequence
        end_ix = i + train_window
        # check if we are beyond the dataset length for this building
        if end_ix > len(building_data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = building_data[i:end_ix, :], building_data[end_ix, 0]
        gt_y = torch.from_numpy(np.array(seq_y)).float()
        n_feat = seq_x.shape[1]
        window = seq_x.reshape(-1, train_window, n_feat)
        window1 = torch.from_numpy(window).float().to(device)
        next_ts = model(window1)
        if np.abs(next_ts.item() - gt_y) >= threshold * next_ts.item():
           # This means I predict an anomaly
           predictions.append(1)
           # Need to substitute
           building_data[end_ix] = next_ts.item()
        else:
           predictions.append(0)
        forecasts.append(next_ts.item())
   return predictions

In [71]:
#FORECASTING
def create_test_sequences(dataframe, time_steps):
  scaler = MinMaxScaler(feature_range=(0,1))
  output = []
  output2=[]
  for building_id, gdf in dataframe.groupby("user_id"):
      gdf[['meter_reading']] = scaler.fit_transform(gdf[['meter_reading']])
      building_data = np.array(gdf[['meter_reading']]).astype(float)
      for i in range(len(building_data)):
        # find the end of this sequence
        end_ix = i + time_steps
        # check if we are beyond the dataset length for this building
        if end_ix > len(building_data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = building_data[i:end_ix, :], building_data[end_ix, 0]
        output.append(seq_x)
        output2.append(seq_y)
      print(len(output))
  return np.stack(output), np.stack(output2)

In [72]:
X_testf, y_testf = create_test_sequences(only_meter_df, 72)

643
1286
1928
2570
3212
3854
4496


In [73]:
X_testf.shape, y_testf.shape

((4496, 72, 1), (4496,))

In [74]:
#FORECASTING
test_loaderf = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_testf).float(), torch.from_numpy(y_testf).float()), batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

In [75]:
modelf = LstmModel(1, 32)
modelf = modelf.to(device) #to_device(model, device)
print(modelf)

LstmModel(
  (lstm): LSTM(1, 32, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [76]:
checkpointf = torch.load("/content/lstm_forec_40_11_06.pth", map_location = torch.device('cpu'))
modelf.load_state_dict(checkpointf)

<All keys matched successfully>

In [77]:
resultsf, wf = testing(modelf, test_loaderf, device)

In [78]:
#FORECASTING
forecast_test = np.concatenate([torch.stack(wf[:-1]).flatten().detach().cpu().numpy(), wf[-1].flatten().detach().cpu().numpy()])

In [79]:
new_dff = pd.DataFrame(y_testf, columns=['y_test'])

In [80]:
new_dff['pred_forec'] = forecast_test

In [81]:
new_dff['abs_loss'] = np.abs(new_dff.y_test - new_dff.pred_forec)
new_dff['rel_loss'] = np.abs((new_dff['pred_forec']-new_dff['y_test'])/(1+new_dff['pred_forec']))

In [82]:
mre = new_dff['rel_loss'].values
#Interquartile threshold
threshold = (np.percentile(np.squeeze(mre), 75)) + 1.5 *((np.percentile(np.squeeze(mre), 75))-(np.percentile(np.squeeze(mre), 25)))
new_dff['threshold'] = threshold
new_dff['predicted_anomaly'] = new_dff['rel_loss'] > new_dff['threshold']

In [83]:
new_dff

,y_test,pred_forec,abs_loss,rel_loss,threshold,predicted_anomaly
0,0.018587,0.275160,0.256573,0.201208,0.21754,False
1,0.226766,0.080117,0.146648,0.135771,0.21754,False
2,0.014870,0.241606,0.226736,0.182615,0.21754,False
3,0.011152,0.084391,0.073238,0.067539,0.21754,False
4,0.003717,0.059731,0.056013,0.052856,0.21754,False
...,...,...,...,...,...,...
4491,0.178378,0.200186,0.021808,0.018170,0.21754,False
4492,0.351351,0.165471,0.185880,0.159489,0.21754,False
4493,0.129730,0.388342,0.258613,0.186274,0.21754,False
4494,0.329730,0.195243,0.134487,0.112519,0.21754,False


In [84]:
vis = new_dff[644:1286]
predicted_anomalies = vis[vis.predicted_anomaly == 1]
print(len(predicted_anomalies))
fig = go.Figure()
fig.add_trace(go.Scatter(x=vis.index, y=vis['y_test'], name='y_test'))
fig.add_trace(go.Scatter(x=vis.index, y=vis['pred_forec'], name='pred'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['y_test'], mode='markers', marker=dict(color='green', size = 8), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='GT VS Predictions')
fig.show()

25
